### Dedupe needs more setup therefore a notebook so i can handle it step by Step


In [1]:
import rdflib

ttl_path = "data/healthcare_graph_Main.ttl"

def extract_dedupe_fields_from_ttl(ttl_path):
    g = rdflib.Graph()
    g.parse(ttl_path)
    
    # Map: {entity_type: set([literal_predicate_names])}
    type_predicate_map = {}
    
    for s in set(g.subjects()):
        # Get type
        types = [str(o) for o in g.objects(s, rdflib.RDF.type)]
        if not types:
            continue
        type_ = types[0].split("/")[-1]  # Or use more sophisticated logic if needed
        
        # Gather literal predicates
        predicates = set()
        for p, o in g.predicate_objects(s):
            if isinstance(o, rdflib.Literal):
                pred_name = p.split("/")[-1] if "/" in str(p) else str(p)
                predicates.add(pred_name)
        if type_ not in type_predicate_map:
            type_predicate_map[type_] = set()
        type_predicate_map[type_].update(predicates)
    
    # Build dedupe.io field definitions per entity type
    dedupe_fields = {}
    for type_, preds in type_predicate_map.items():
        dedupe_fields[type_] = [
            {'field': pred, 'type': 'String'} for pred in preds if pred.lower() != "identifier"
]

    return dedupe_fields

fields_per_type = extract_dedupe_fields_from_ttl("data/healthcare_graph_Main.ttl")
for entity_type, fields in fields_per_type.items():
    print(f"Entity type: {entity_type}")
    print(fields)
    print()

Entity type: Person
[{'field': 'jobTitle', 'type': 'String'}, {'field': 'name', 'type': 'String'}, {'field': 'birthDate', 'type': 'String'}, {'field': 'email', 'type': 'String'}, {'field': 'gender', 'type': 'String'}, {'field': 'knowsLanguage', 'type': 'String'}]

Entity type: Department
[{'field': 'name', 'type': 'String'}]

Entity type: PostalAddress
[{'field': 'streetAddress', 'type': 'String'}, {'field': 'addressLocality', 'type': 'String'}, {'field': 'addressCountry', 'type': 'String'}, {'field': 'postalCode', 'type': 'String'}]

Entity type: ContactPoint
[{'field': 'contactType', 'type': 'String'}, {'field': 'telephone', 'type': 'String'}, {'field': 'availableLanguage', 'type': 'String'}, {'field': 'email', 'type': 'String'}, {'field': 'faxNumber', 'type': 'String'}]

Entity type: MedicalOrganization
[{'field': 'name', 'type': 'String'}]



In [ ]:
from modular_methods.graphToText_utils import kg_to_dedupe_dict
g = rdflib.Graph() # Main graph
g1 = rdflib.Graph() # Replaced low graph
g2 = rdflib.Graph() # Train graph
g3 = rdflib.Graph() # Replaced medium graph
g4 = rdflib.Graph() # Replaced high graph
g5 = rdflib.Graph() # Struct low graph
g6 = rdflib.Graph() # Struct high graph
g7 = rdflib.Graph() # Struct train_low graph
g8 = rdflib.Graph() # Struct train_high graph

g.parse("data/healthcare_graph_Main.ttl")
g1.parse("data/healthcare_graph_relation.ttl")

g2.parse("data/healthcare_graph_train.ttl")
g3.parse("data/healthcare_graph_train_relation.ttl")
g4.parse("data/healthcare_graph_replaced_high.ttl")
g5.parse("data/healthcare_graph_struct_low.ttl")
g6.parse("data/healthcare_graph_struct_high.ttl")
g7.parse("data/healthcare_graph_train_struct_low.ttl")
g8.parse("data/healthcare_graph_train_struct_high.ttl")
dict_main = kg_to_dedupe_dict(g)

train_dict = kg_to_dedupe_dict(g2)
dict_relation = kg_to_dedupe_dict(g1)
dict_relation_train = kg_to_dedupe_dict(g3)
dict_high = kg_to_dedupe_dict(g4)
dict_struct_low = kg_to_dedupe_dict(g5)
dict_struct_high = kg_to_dedupe_dict(g6)
dict_struct_train_low = kg_to_dedupe_dict(g7)
dict_struct_train_high = kg_to_dedupe_dict(g8)   

In [ ]:
import json
with open("data/dedupe_data/dict_main.json", "w") as f:
    json.dump(dict_main, f, indent=2)
with open("data/dedupe_data/dict_high.json", "w") as f:
    json.dump(dict_high, f, indent=2)
with open("data/dedupe_data/dict_relation.json", "w") as f:
    json.dump(dict_relation, f, indent=2)
with open("data/dedupe_data/dict_relation_train.json", "w") as f:
    json.dump(dict_relation_train, f, indent=2)
with open("data/dedupe_data/train_dict.json", "w") as f:
    json.dump(train_dict, f, indent=2)
with open("data/dedupe_data/dict_struct_low.json", "w") as f:
    json.dump(dict_struct_low, f, indent=2)
with open("data/dedupe_data/dict_struct_high.json", "w") as f:
    json.dump(dict_struct_high, f, indent=2)
with open("data/dedupe_data/dict_struct_train_low.json", "w") as f:
    json.dump(dict_struct_train_low, f, indent=2)
with open("data/dedupe_data/dict_struct_train_high.json", "w") as f:
    json.dump(dict_struct_train_high, f, indent=2)
print("Dedupe data saved to JSON files.")

Dedupe data saved to JSON files.


## Ground truth for test data

In [ ]:
import pandas as pd
import os

# List your test golden standard files here
test_gs_files = [
    "data/test_golden_standard_high.csv",
    "data/test_golden_standard_struct_low.csv",
    "data/test_golden_standard_struct_high.csv",
    "data/test_golden_standard_relation.csv",
]

output_dir = "data/dedupe_data/ground_truths_test"
os.makedirs(output_dir, exist_ok=True)

for gs_file in test_gs_files:
    golden_standard = pd.read_csv(gs_file)
    golden_standard_subset_var = golden_standard[['original_id', 'duplicate_id', 'entity_type','variation_type']]

    ground_truth_test = {}

    for entity in golden_standard_subset_var['entity_type'].unique():
        df_filtered = golden_standard_subset_var[golden_standard_subset_var['entity_type'] == entity]
        entity_for_uri = "Person" if entity in ["Person", "HealthcarePersonnel"] else entity
        key = "Person" if entity in ["Person", "HealthcarePersonnel"] else entity
        if key not in ground_truth_test:
            ground_truth_test[key] = []
        ground_truth_test[key].extend([
            (f"http://example.org/{entity_for_uri}/{row['original_id']}", 
             f"http://example.org/{entity_for_uri}/{row['duplicate_id']}")
            for _, row in df_filtered.iterrows()
        ])
    
    # Save each GT as a pickle or CSV for later use
    base = os.path.splitext(os.path.basename(gs_file))[0]
    for entity, pairs in ground_truth_test.items():
        # Save as CSV
        pd.DataFrame(pairs, columns=["uri1", "uri2"]).to_csv(
            f"{output_dir}/{base}_GT_{entity}.csv", index=False
        )
        # Optionally, save as pickle for fast loading
        # pd.DataFrame(pairs, columns=["uri1", "uri2"]).to_pickle(
        #     f"{output_dir}/{base}_GT_{entity}.pkl"
        # )
    print(f"Saved GT files for {gs_file}")

Saved GT files for data/test_golden_standard_high.csv
Saved GT files for data/test_golden_standard_struct_low.csv
Saved GT files for data/test_golden_standard_struct_high.csv


### Perform the manual labelling on train data NOT on test data

In [ ]:

train_gs_files = [
    "data/train_golden_standard_duplicates_updated.csv",
    "data/train_golden_standard_struct_high.csv",
    "data/train_golden_standard_struct_low.csv",
    "data/train_golden_standard_relation.csv",
]

output_dir = "data/dedupe_data/ground_truths_train"
os.makedirs(output_dir, exist_ok=True)

for gs_file in train_gs_files:
    golden_standard = pd.read_csv(gs_file)
    golden_standard_subset_var = golden_standard[['original_id', 'duplicate_id', 'entity_type','variation_type']]

    ground_truth_train = {}

    # Mapping to Person for both Person and HealthcarePersonnel
    for entity in golden_standard_subset_var['entity_type'].unique():
        df_filtered = golden_standard_subset_var[golden_standard_subset_var['entity_type'] == entity]
        entity_for_uri = "Person" if entity in ["Person", "HealthcarePersonnel"] else entity
        # Use 'Person' as the key for both types
        key = "Person" if entity in ["Person", "HealthcarePersonnel"] else entity
        if key not in ground_truth_train:
            ground_truth_train[key] = []
        ground_truth_train[key].extend([
            (f"http://example.org/{entity_for_uri}/{row['original_id']}", 
            f"http://example.org/{entity_for_uri}/{row['duplicate_id']}")
            for _, row in df_filtered.iterrows()
        ])

    # Save each GT as a pickle or CSV for later use
    base = os.path.splitext(os.path.basename(gs_file))[0]
    for entity, pairs in ground_truth_test.items():
        # Save as CSV
        pd.DataFrame(pairs, columns=["uri1", "uri2"]).to_csv(
            f"{output_dir}/{base}_GT_{entity}.csv", index=False
        )
        # Optionally, save as pickle for fast loading
        # pd.DataFrame(pairs, columns=["uri1", "uri2"]).to_pickle(
        #     f"{output_dir}/{base}_GT_{entity}.pkl"
        # )
    print(f"Saved GT files for {gs_file}")



# person_variations = golden_standard_subset_var[
#     (golden_standard_subset_var['entity_type'] == 'Person') | 
#     (golden_standard_subset_var['entity_type'] == 'HealthcarePersonnel')
# ]

# person_variations

Saved GT files for data/train_golden_standard_duplicates_updated.csv
Saved GT files for data/train_golden_standard_struct_high.csv
Saved GT files for data/train_golden_standard_struct_low.csv


### Matches per variation type for training

In [9]:
# GT_Address = pd.read_csv('data/dedupe_data/ground_truths_train/train_golden_standard_duplicates_updated_GT_Address.csv')
# GT_Person = pd.read_csv('data/dedupe_data/ground_truths_train/train_golden_standard_duplicates_updated_GT_Person.csv')
# GT_HCO = pd.read_csv('data/dedupe_data/ground_truths_train/train_golden_standard_duplicates_updated_GT_HealthcareOrganization.csv')
# GT_SD = pd.read_csv('data/dedupe_data/ground_truths_train/train_golden_standard_duplicates_updated_GT_ServiceDepartment.csv')
# GT_CP = pd.read_csv('data/dedupe_data/ground_truths_train/train_golden_standard_duplicates_updated_GT_ContactPoint.csv')

# noise = "low"
# GT_Address_struct = pd.read_csv(f'data/dedupe_data/ground_truths_train/train_golden_standard_struct_{noise}_GT_Address.csv')
# GT_Person_struct = pd.read_csv(f'data/dedupe_data/ground_truths_train/train_golden_standard_struct_{noise}_GT_Person.csv')
# GT_HCO_struct = pd.read_csv(f'data/dedupe_data/ground_truths_train/train_golden_standard_struct_{noise}_GT_HealthcareOrganization.csv')
# GT_SD_struct = pd.read_csv(f'data/dedupe_data/ground_truths_train/train_golden_standard_struct_{noise}_GT_ServiceDepartment.csv')
# GT_CP_struct = pd.read_csv(f'data/dedupe_data/ground_truths_train/train_golden_standard_struct_{noise}_GT_ContactPoint.csv')

# Get 7 examples per variation type
output_dir = "data/dedupe_data/ground_truths_per_variation_type"
os.makedirs(output_dir, exist_ok=True)
for file in train_gs_files:
    golden_standard = pd.read_csv(file)
    golden_standard_subset_var = golden_standard[['original_id', 'duplicate_id', 'entity_type','variation_type']]
    

    entity_subsets = {
        "Person": golden_standard_subset_var[
            (golden_standard_subset_var['entity_type'] == 'Person') | 
            (golden_standard_subset_var['entity_type'] == 'HealthcarePersonnel')
        ],
        "Address": golden_standard_subset_var[
            golden_standard_subset_var['entity_type'] == 'Address'
        ],
        "HealthcareOrganization": golden_standard_subset_var[
            golden_standard_subset_var['entity_type'] == 'HealthcareOrganization'
        ],
        "ServiceDepartment": golden_standard_subset_var[
            golden_standard_subset_var['entity_type'] == 'ServiceDepartment'
        ],
        "ContactPoint": golden_standard_subset_var[
            golden_standard_subset_var['entity_type'] == 'ContactPoint'
        ]
    }

       # For each entity type, get up to N pairs per variation type
    N = 10  # or whatever number you want
    gt_by_entity = {}
    for entity_type, subset in entity_subsets.items():
        gt_by_entity[entity_type] = []
        for variation_type in subset['variation_type'].unique():
            variation_pairs = subset[subset['variation_type'] == variation_type]
            selected_pairs = variation_pairs.head(N)
            for _, row in selected_pairs.iterrows():
                # Use correct URI prefix for Person/HealthcarePersonnel
                entity_for_uri = "Person" if entity_type == "Person" else entity_type
                gt_by_entity[entity_type].append((
                    f"http://example.org/{entity_for_uri}/{row['original_id']}",
                    f"http://example.org/{entity_for_uri}/{row['duplicate_id']}"
                ))
        print(f"{entity_type}: {len(gt_by_entity[entity_type])} pairs selected")
    file_split = file.split("/")[-1].split(".")[0]
    # Optionally, save to CSV
    for entity_type, pairs in gt_by_entity.items():
        pd.DataFrame(pairs, columns=["uri1", "uri2"]).to_csv(
            f"{output_dir}/{file_split}_GT_{entity_type}.csv", index=False)
    







Person: 60 pairs selected
Address: 40 pairs selected
HealthcareOrganization: 20 pairs selected
ServiceDepartment: 20 pairs selected
ContactPoint: 30 pairs selected
Person: 10 pairs selected
Address: 10 pairs selected
HealthcareOrganization: 10 pairs selected
ServiceDepartment: 10 pairs selected
ContactPoint: 10 pairs selected
Person: 10 pairs selected
Address: 10 pairs selected
HealthcareOrganization: 0 pairs selected
ServiceDepartment: 0 pairs selected
ContactPoint: 10 pairs selected


### Distinct for training

In [10]:
import random
import pandas as pd
import os

def sample_distinct_pairs(golden_standard_subset, ground_truth_pairs, entity_type, n_neg=3):
    """Sample distinct pairs for dedupe.io (negatives), avoiding ground truth."""
    filtered = golden_standard_subset[golden_standard_subset['entity_type'] == entity_type]
    all_ids1 = filtered['original_id'].unique()
    all_ids2 = filtered['duplicate_id'].unique()
    gt_set = set(ground_truth_pairs)
    distinct_pairs = set()
    attempts = 0
    max_attempts = len(gt_set) * n_neg * 10  # Prevent infinite loop
    
    while len(distinct_pairs) < len(gt_set) * n_neg and attempts < max_attempts:
        a, b = random.choice(all_ids1), random.choice(all_ids2)
        if (a, b) not in gt_set and (b, a) not in gt_set and (a, b) not in distinct_pairs:
            distinct_pairs.add((f"http://example.org/{entity_type}/{a}", f"http://example.org/{entity_type}/{b}"))
        attempts += 1
    return list(distinct_pairs)

# List your golden files (train or test)
golden_files = [
    "data/train_golden_standard_duplicates_updated.csv",
    "data/train_golden_standard_struct_high.csv",
    "data/train_golden_standard_struct_low.csv",
    # add test files if needed
]

output_dir = "data/dedupe_data/ground_truths_distinct_train"
os.makedirs(output_dir, exist_ok=True)

for file in golden_files:
    golden_standard = pd.read_csv(file)
    golden_standard_subset = golden_standard[['original_id', 'duplicate_id', 'entity_type','variation_type']]
    # Build ground truth for each entity type
    ground_truth = {}
    for entity in golden_standard_subset['entity_type'].unique():
        df_filtered = golden_standard_subset[golden_standard_subset['entity_type'] == entity]
        entity_for_uri = "Person" if entity in ["Person", "HealthcarePersonnel"] else entity
        key = "Person" if entity in ["Person", "HealthcarePersonnel"] else entity
        if key not in ground_truth:
            ground_truth[key] = []
        ground_truth[key].extend([
            (f"http://example.org/{entity_for_uri}/{row['original_id']}", 
             f"http://example.org/{entity_for_uri}/{row['duplicate_id']}")
            for _, row in df_filtered.iterrows()
        ])
    # Now sample and save distinct pairs for each entity type
    for entity_type in ground_truth.keys():
        negatives = sample_distinct_pairs(
            golden_standard_subset, ground_truth[entity_type], entity_type, n_neg=3
        )
        base = os.path.splitext(os.path.basename(file))[0]
        pd.DataFrame(negatives, columns=["uri1", "uri2"]).to_csv(
            f"{output_dir}/{base}_DISTINCT_{entity_type}.csv", index=False
        )
        print(f"Saved {len(negatives)} distinct pairs for {entity_type} in {base}")

Saved 768 distinct pairs for Address in train_golden_standard_duplicates_updated
Saved 84 distinct pairs for HealthcareOrganization in train_golden_standard_duplicates_updated
Saved 684 distinct pairs for ServiceDepartment in train_golden_standard_duplicates_updated
Saved 2619 distinct pairs for Person in train_golden_standard_duplicates_updated
Saved 768 distinct pairs for ContactPoint in train_golden_standard_duplicates_updated
Saved 963 distinct pairs for Address in train_golden_standard_struct_high
Saved 963 distinct pairs for ContactPoint in train_golden_standard_struct_high
Saved 2739 distinct pairs for Person in train_golden_standard_struct_high
Saved 105 distinct pairs for HealthcareOrganization in train_golden_standard_struct_high
Saved 858 distinct pairs for ServiceDepartment in train_golden_standard_struct_high
Saved 963 distinct pairs for Address in train_golden_standard_struct_low
Saved 963 distinct pairs for ContactPoint in train_golden_standard_struct_low
Saved 2739 dist

In [9]:
# Map of entity type to dedupe.io fields
ENTITY_FIELDS = {
    "Person": [
        {'field': 'knowsLanguage', 'type': 'String'},
        {'field': 'jobTitle', 'type': 'String'},
        {'field': 'name', 'type': 'String'},
        {'field': 'birthDate', 'type': 'String'},
        {'field': 'gender', 'type': 'String'},
        {'field': 'email', 'type': 'String'}
    ],
    "ContactPoint": [
        {'field': 'faxNumber', 'type': 'String'},
        {'field': 'availableLanguage', 'type': 'String'},
        {'field': 'telephone', 'type': 'String'},
        {'field': 'email', 'type': 'String'},
        {'field': 'contactType', 'type': 'String'}
    ],
    "Department": [
        {'field': 'name', 'type': 'String'}
    ],
    "PostalAddress": [
        {'field': 'addressLocality', 'type': 'String'},
        {'field': 'streetAddress', 'type': 'String'},
        {'field': 'postalCode', 'type': 'String'},
        {'field': 'addressCountry', 'type': 'String'}
    ],
    "MedicalOrganization": [
        {'field': 'name', 'type': 'String'}
    ]
}

import dedupe
import dedupe.variables

ENTITY_FIELDS_dedup= {"Person": [
        dedupe.variables.String("knowsLanguage"),
        dedupe.variables.String("jobTitle"),
        dedupe.variables.String("name"),
        dedupe.variables.String("birthDate"),
        dedupe.variables.String("gender"),
        dedupe.variables.String("email")
    ],
    "ContactPoint": [
        dedupe.variables.String("faxNumber"),
        dedupe.variables.String("availableLanguage"),
        dedupe.variables.String("telephone"),
        dedupe.variables.String("email"),
        dedupe.variables.String("contactType")
    ],
    "Department": [
        dedupe.variables.String("name")
    ],
    "PostalAddress": [
        dedupe.variables.String("addressLocality"),
        dedupe.variables.String("streetAddress"),
        dedupe.variables.String("postalCode"),
        dedupe.variables.String("addressCountry")
    ],
    "MedicalOrganization": [
        dedupe.variables.String("name")
    ]}





In [ ]:
def filter_entity(d, entity_type):
    search_str = f"/{entity_type}/"
    return {k: v for k, v in d.items() if search_str in k}


person_main = filter_entity(dict_main, "Person")
person_high = filter_entity(dict_high, "Person")
person_struct_low = filter_entity(dict_struct_low, "Person")
person_struct_high = filter_entity(dict_struct_high, "Person")
train_person_dict1 = filter_entity(train_dict, "Person")
train_person_low = filter_entity(dict_struct_train_low, "Person")
train_person_high = filter_entity(dict_struct_train_high, "Person")

contactpoint_main = filter_entity(dict_main, "ContactPoint")
contactpoint_high = filter_entity(dict_high, "ContactPoint")
contactpoint_struct_low = filter_entity(dict_struct_low, "ContactPoint")
contactpoint_struct_high = filter_entity(dict_struct_high, "ContactPoint")
train_contactpoint_dict1 = filter_entity(train_dict, "ContactPoint")
train_contactpoint_low = filter_entity(dict_struct_train_low, "ContactPoint")
train_contactpoint_high = filter_entity(dict_struct_train_high, "ContactPoint")

address_main = filter_entity(dict_main, "Address")
address_high = filter_entity(dict_high, "Address")
address_struct_low = filter_entity(dict_struct_low, "Address")
address_struct_high = filter_entity(dict_struct_high, "Address")
train_address_dict1 = filter_entity(train_dict, "Address")
train_address_low = filter_entity(dict_struct_train_low, "Address")
train_address_high = filter_entity(dict_struct_train_high, "Address")

hco_main = filter_entity(dict_main, "HealthcareOrganization")
hco_high = filter_entity(dict_high, "HealthcareOrganization")
hco_struct_low = filter_entity(dict_struct_low, "HealthcareOrganization")
hco_struct_high = filter_entity(dict_struct_high, "HealthcareOrganization")
train_hco_dict1 = filter_entity(train_dict, "HealthcareOrganization")
train_hco_low = filter_entity(dict_struct_train_low, "HealthcareOrganization")
train_hco_high = filter_entity(dict_struct_train_high, "HealthcareOrganization")

sd_main = filter_entity(dict_main, "ServiceDepartment")
sd_high = filter_entity(dict_high, "ServiceDepartment")
sd_struct_low = filter_entity(dict_struct_low, "ServiceDepartment")
sd_struct_high = filter_entity(dict_struct_high, "ServiceDepartment")
train_sd_dict1 = filter_entity(train_dict, "ServiceDepartment")
train_sd_low = filter_entity(dict_struct_train_low, "ServiceDepartment")
train_sd_high = filter_entity(dict_struct_train_high, "ServiceDepartment")


def replace_nan_in_dict(d):
    import numpy as np
    for k, v in d.items():
        if isinstance(v, dict):
            d[k] = replace_nan_in_dict(v)
        elif (isinstance(v, float) and np.isnan(v)) or (isinstance(v, str) and v.lower() == 'nan'):
            d[k] = ''
    return d

# Replace NaN values in all dictionaries
person_struct_low = replace_nan_in_dict(person_struct_low)
person_struct_high = replace_nan_in_dict(person_struct_high)
contactpoint_struct_low = replace_nan_in_dict(contactpoint_struct_low)
contactpoint_struct_high = replace_nan_in_dict(contactpoint_struct_high)
address_struct_low = replace_nan_in_dict(address_struct_low)
address_struct_high = replace_nan_in_dict(address_struct_high)
hco_struct_low = replace_nan_in_dict(hco_struct_low)
hco_struct_high = replace_nan_in_dict(hco_struct_high)
sd_struct_low = replace_nan_in_dict(sd_struct_low)
sd_struct_high = replace_nan_in_dict(sd_struct_high)
train_person_low = replace_nan_in_dict(train_person_low)
train_person_high = replace_nan_in_dict(train_person_high)
train_contactpoint_low = replace_nan_in_dict(train_contactpoint_low)
train_contactpoint_high = replace_nan_in_dict(train_contactpoint_high)
train_address_low = replace_nan_in_dict(train_address_low)
train_address_high = replace_nan_in_dict(train_address_high)
train_hco_low = replace_nan_in_dict(train_hco_low)
train_hco_high = replace_nan_in_dict(train_hco_high)
train_sd_low = replace_nan_in_dict(train_sd_low)
train_sd_high = replace_nan_in_dict(train_sd_high)


In [21]:
train_hco_high

{'http://example.org/HealthcareOrganization/ba1c6582-628b-41c7-957e-d65369bbfc78': {'name': ''},
 'http://example.org/HealthcareOrganization/6a294d2d-0afe-42dc-b1ad-ea8a8f679cb2': {'name': ''},
 'http://example.org/HealthcareOrganization/6558d20b-b614-48de-a48a-44a3ae383343': {'name': ''},
 'http://example.org/HealthcareOrganization/5eca4e17-0652-4819-b7ea-e133d20e49a5': {'name': ''},
 'http://example.org/HealthcareOrganization/7e4abe74-c3f7-45f6-812b-1e9b4ac50911': {'name': ''},
 'http://example.org/HealthcareOrganization/8697ea93-5041-4718-ac4c-d9ca82b88df0': {'name': ''},
 'http://example.org/HealthcareOrganization/004e0b1f-a990-431d-9252-dc9531866aa1': {'name': ''},
 'http://example.org/HealthcareOrganization/c02c1a44-c971-48bb-9e36-06d3d2a21afe': {'name': ''},
 'http://example.org/HealthcareOrganization/75572bae-67d5-434a-b604-0948bf0deab7': {'name': ''},
 'http://example.org/HealthcareOrganization/4212571c-7d43-4010-b0fc-2246d75a3d57': {'name': ''},
 'http://example.org/Healthcar

In [34]:
import csv

def pairs_to_records(pair_list, dict_main, train_dict):
    # Accepts DataFrame or list of tuples
    if hasattr(pair_list, "values"):  # DataFrame
        pair_iter = pair_list[["uri1", "uri2"]].values
    else:
        pair_iter = pair_list
    return [
        (dict_main[a], train_dict[b])
        for a, b in pair_iter
        if a in dict_main and b in train_dict
    ]


def write_pairs_to_csv(pairs, filename, expected_fields=None):
    """Write pairs of records to a CSV file for manual editing."""
    if not pairs:
        print("No pairs to write.")
        return

    # If not provided, infer from all pairs
    if expected_fields is None:
        all_keys_left = set()
        all_keys_right = set()
        for rec1, rec2 in pairs:
            all_keys_left.update(rec1.keys())
            all_keys_right.update(rec2.keys())
        expected_fields = sorted(all_keys_left | all_keys_right)  # union

    fieldnames = ['label'] + [f'left_{k}' for k in expected_fields] + [f'right_{k}' for k in expected_fields]

    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for rec1, rec2 in pairs:
            row = {'label': ''}
            for k in expected_fields:
                row[f"left_{k}"] = rec1.get(k, '')
                row[f"right_{k}"] = rec2.get(k, '')
            writer.writerow(row)

def read_pairs_from_csv(filename):
    """Read pairs of records from a CSV file (after manual annotation)."""
    pairs = []
    labels = []
    with open(filename, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            rec1 = {k[5:]: v for k, v in row.items() if k.startswith('left_')}
            rec2 = {k[6:]: v for k, v in row.items() if k.startswith('right_')}
            label = row.get('label', '')
            pairs.append((rec1, rec2))
            labels.append(label)
    return pairs, labels

GT_Person = pd.read_csv('data/dedupe_data/ground_truths_per_variation_type/train_golden_standard_duplicates_updated_GT_Person.csv')
GT_Address = pd.read_csv('data/dedupe_data/ground_truths_per_variation_type/train_golden_standard_duplicates_updated_GT_Address.csv')
GT_HCO = pd.read_csv('data/dedupe_data/ground_truths_per_variation_type/train_golden_standard_duplicates_updated_GT_HealthcareOrganization.csv')
GT_SD = pd.read_csv('data/dedupe_data/ground_truths_per_variation_type/train_golden_standard_duplicates_updated_GT_ServiceDepartment.csv')
GT_CP = pd.read_csv('data/dedupe_data/ground_truths_per_variation_type/train_golden_standard_duplicates_updated_GT_ContactPoint.csv')

DP_Person = pd.read_csv('data/dedupe_data/ground_truths_distinct_train/train_golden_standard_duplicates_updated_DISTINCT_Person.csv')
DP_Address = pd.read_csv('data/dedupe_data/ground_truths_distinct_train/train_golden_standard_duplicates_updated_DISTINCT_Address.csv')
DP_HCO = pd.read_csv('data/dedupe_data/ground_truths_distinct_train/train_golden_standard_duplicates_updated_DISTINCT_HealthcareOrganization.csv')
DP_SD = pd.read_csv('data/dedupe_data/ground_truths_distinct_train/train_golden_standard_duplicates_updated_DISTINCT_ServiceDepartment.csv')
DP_CP = pd.read_csv('data/dedupe_data/ground_truths_distinct_train/train_golden_standard_duplicates_updated_DISTINCT_ContactPoint.csv')

noise = "high"
if noise == "low":
    train_dict = dict_struct_train_low
    GT_Person = pd.read_csv(f'data/dedupe_data/ground_truths_per_variation_type/train_golden_standard_struct_{noise}_GT_Person.csv')
    GT_Address = pd.read_csv(f'data/dedupe_data/ground_truths_per_variation_type/train_golden_standard_struct_{noise}_GT_Address.csv')
    GT_CP = pd.read_csv(f'data/dedupe_data/ground_truths_per_variation_type/train_golden_standard_struct_{noise}_GT_ContactPoint.csv')
    DP_Person = pd.read_csv(f'data/dedupe_data/ground_truths_distinct_train/train_golden_standard_struct_{noise}_DISTINCT_Person.csv')
    DP_Address = pd.read_csv(f'data/dedupe_data/ground_truths_distinct_train/train_golden_standard_struct_{noise}_DISTINCT_Address.csv')
    DP_CP = pd.read_csv(f'data/dedupe_data/ground_truths_distinct_train/train_golden_standard_struct_{noise}_DISTINCT_ContactPoint.csv')

    matches_Person = pairs_to_records(GT_Person, dict_main, dict_struct_train_low)
    distinct_Person = pairs_to_records(DP_Person, dict_main, dict_struct_train_low)

    matches_ContactPoint = pairs_to_records(GT_CP, dict_main, dict_struct_train_low)
    distinct_ContactPoint = pairs_to_records(DP_CP, dict_main, dict_struct_train_low)

    matches_Address = pairs_to_records(GT_Address, dict_main, dict_struct_train_low)
    distinct_Address = pairs_to_records(DP_Address, dict_main, dict_struct_train_low)

        #Example usage:
    for matches , distinct, entity_type in [
        (matches_Person, distinct_Person, "Person"),
        (matches_ContactPoint, distinct_ContactPoint, "ContactPoint"),
        (matches_Address, distinct_Address, "Address")
    ]:
        filename = f"data/dedupe_data/labelling_struct/matches_for_labelling_{entity_type}_{noise}.csv"
        write_pairs_to_csv(matches, filename)
        write_pairs_to_csv(distinct, filename.replace("matches_for_labelling", "distinct_pairs_for_labelling"))
        print(f"Wrote {len(matches)} matches and {len(distinct)} distinct pairs to {filename}")
    

if noise == "high":
    train_dict = dict_struct_train_high
    GT_Person = pd.read_csv(f'data/dedupe_data/ground_truths_per_variation_type/train_golden_standard_struct_{noise}_GT_Person.csv')
    GT_Address = pd.read_csv(f'data/dedupe_data/ground_truths_per_variation_type/train_golden_standard_struct_{noise}_GT_Address.csv')
    GT_CP = pd.read_csv(f'data/dedupe_data/ground_truths_per_variation_type/train_golden_standard_struct_{noise}_GT_ContactPoint.csv')
    GT_HCO = pd.read_csv(f'data/dedupe_data/ground_truths_per_variation_type/train_golden_standard_struct_{noise}_GT_HealthcareOrganization.csv')
    GT_SD = pd.read_csv(f'data/dedupe_data/ground_truths_per_variation_type/train_golden_standard_struct_{noise}_GT_ServiceDepartment.csv')
    DP_Person = pd.read_csv(f'data/dedupe_data/ground_truths_distinct_train/train_golden_standard_struct_{noise}_DISTINCT_Person.csv')
    DP_Address = pd.read_csv(f'data/dedupe_data/ground_truths_distinct_train/train_golden_standard_struct_{noise}_DISTINCT_Address.csv')
    DP_CP = pd.read_csv(f'data/dedupe_data/ground_truths_distinct_train/train_golden_standard_struct_{noise}_DISTINCT_ContactPoint.csv')
    DP_HCO = pd.read_csv(f'data/dedupe_data/ground_truths_distinct_train/train_golden_standard_struct_{noise}_DISTINCT_HealthcareOrganization.csv')
    DP_SD = pd.read_csv(f'data/dedupe_data/ground_truths_distinct_train/train_golden_standard_struct_{noise}_DISTINCT_ServiceDepartment.csv')

    matches_Person = pairs_to_records(GT_Person, dict_main, dict_struct_train_high)
    distinct_Person = pairs_to_records(DP_Person, dict_main, dict_struct_train_high)

    matches_ContactPoint = pairs_to_records(GT_CP, dict_main, dict_struct_train_high)
    distinct_ContactPoint = pairs_to_records(DP_CP, dict_main, dict_struct_train_high)

    matches_Address = pairs_to_records(GT_Address, dict_main, dict_struct_train_high)
    distinct_Address = pairs_to_records(DP_Address, dict_main, dict_struct_train_high)

    matches_HCO = pairs_to_records(GT_HCO, dict_main, dict_struct_train_high)
    distinct_HCO = pairs_to_records(DP_HCO, dict_main, dict_struct_train_high)

    matches_SD = pairs_to_records(GT_SD, dict_main, dict_struct_train_high)
    distinct_SD = pairs_to_records(DP_SD, dict_main, dict_struct_train_high)

        #Example usage:
    for matches , distinct, entity_type in [
        (matches_Person, distinct_Person, "Person"),
        (matches_ContactPoint, distinct_ContactPoint, "ContactPoint"),
        (matches_Address, distinct_Address, "Address"),
        (matches_HCO, distinct_HCO, "HealthcareOrganization"),
        (matches_SD, distinct_SD, "ServiceDepartment")
    ]:
        filename = f"data/dedupe_data/labelling_struct/matches_for_labelling_{entity_type}_{noise}.csv"
        write_pairs_to_csv(matches, filename)
        write_pairs_to_csv(distinct, filename.replace("matches_for_labelling", "distinct_pairs_for_labelling"))
        print(f"Wrote {len(matches)} matches and {len(distinct)} distinct pairs to {filename}")
    



# matches_Person = pairs_to_records(GT_Person, dict_main, train_dict)
# distinct_Person = pairs_to_records(DP_Person, dict_main, train_dict)

# matches_ContactPoint = pairs_to_records(GT_CP, dict_main, train_dict)
# distinct_ContactPoint = pairs_to_records(DP_CP, dict_main, train_dict)

# matches_Address = pairs_to_records(GT_Address, dict_main, train_dict)
# distinct_Address = pairs_to_records(DP_Address, dict_main, train_dict)

# matches_HCO = pairs_to_records(GT_HCO, dict_main, train_dict)
# distinct_HCO = pairs_to_records(DP_HCO, dict_main, train_dict)

# matches_SD = pairs_to_records(GT_SD, dict_main, train_dict)
# distinct_SD = pairs_to_records(DP_SD, dict_main, train_dict)




# #Example usage:
# for matches , distinct, entity_type in [
#     (matches_Person, distinct_Person, "Person"),
#     (matches_ContactPoint, distinct_ContactPoint, "ContactPoint"),
#     (matches_Address, distinct_Address, "Address"),
#     (matches_HCO, distinct_HCO, "HealthcareOrganization"),
#     (matches_SD, distinct_SD, "ServiceDepartment")
# ]:
#     filename = f"data/dedupe_data/labelling_train/matches_for_labelling_{entity_type}.csv"
#     write_pairs_to_csv(matches, filename)
#     write_pairs_to_csv(distinct, filename.replace("matches_for_labelling", "distinct_pairs_for_labelling"))
#     print(f"Wrote {len(matches)} matches and {len(distinct)} distinct pairs to {filename}")


Wrote 10 matches and 2739 distinct pairs to data/dedupe_data/labelling_struct/matches_for_labelling_Person_high.csv
Wrote 10 matches and 963 distinct pairs to data/dedupe_data/labelling_struct/matches_for_labelling_ContactPoint_high.csv
Wrote 10 matches and 963 distinct pairs to data/dedupe_data/labelling_struct/matches_for_labelling_Address_high.csv
Wrote 10 matches and 105 distinct pairs to data/dedupe_data/labelling_struct/matches_for_labelling_HealthcareOrganization_high.csv
Wrote 10 matches and 858 distinct pairs to data/dedupe_data/labelling_struct/matches_for_labelling_ServiceDepartment_high.csv
